In [37]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Carregando os dados
base = pd.read_csv('C:/Users/milen/OneDrive/Documentos/TCC/Bases/weater_base.csv')

# Substituindo valores NaN por 0
base.fillna(base.mean(), inplace=True)

# Garanta que a coluna 'Data' é do tipo datetime
base['Data'] = pd.to_datetime(base['Data'])

# Defina a coluna 'Data' como o índice
base.set_index('Data', inplace=True)

# Divisão dos Dados
train_data = base[base.index < '2023-09-01']
test_data = base[base.index >= '2023-09-01']

# Ajuste do modelo ARIMA
p, d, q = 4, 5, 6  # Parâmetros do modelo ARIMA (ajuste conforme necessário)
# Ajuste do modelo ARIMA com os parâmetros p, d, q
model = ARIMA(train_data['Preco_Real'], order=(p, d, q))
model_fit = model.fit()

# Previsões
predictions = model_fit.forecast(steps=len(test_data))

# Avaliação do Modelo
rmse = np.sqrt(mean_squared_error(test_data['Preco_Real'], predictions))
mae = mean_absolute_error(test_data['Preco_Real'], predictions)
mape = np.mean(np.abs(predictions - test_data['Preco_Real']) / np.abs(test_data['Preco_Real'])) * 100

print(f"RMSE: {rmse}")
print(f"MAE: {mae}")
print(f"MAPE: {mape}%")

# Visualização dos Resultados
plt.figure(figsize=(12, 6))
sns.lineplot(x=test_data.index, y=test_data['Preco_Real'], label="Real")
sns.lineplot(x=test_data.index, y=predictions, label="Previsão")
plt.xlabel("Data")
plt.ylabel("Preço Real")
plt.title("Previsão vs. Real")
plt.legend()
plt.show()


C:\Users\milen\AppData\Local\Temp\ipykernel_25160\3396176432.py:16: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  base.fillna(base.mean(), inplace=True)
c:\Users\milen\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\milen\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\milen\AppData\Local\Programs\Python\Pyth

LinAlgError: LU decomposition error.

In [18]:
from pmdarima import auto_arima

# Executa o auto_arima para encontrar os melhores parâmetros
auto_model = auto_arima(train_data['Preco_Real'], seasonal=True, m=12, trace=True, error_action='ignore', suppress_warnings=True)

# Ajuste do modelo com os melhores parâmetros encontrados
model_fit = auto_model.fit(train_data['Preco_Real'])

# Previsões
predictions = model_fit.predict(n_periods=len(test_data))

# Código para avaliação do modelo e visualização continua igual...


Performing stepwise search to minimize aic
 ARIMA(2,1,2)(1,0,1)[12] intercept   : AIC=31566.406, Time=4.83 sec
 ARIMA(0,1,0)(0,0,0)[12] intercept   : AIC=31581.860, Time=0.04 sec
 ARIMA(1,1,0)(1,0,0)[12] intercept   : AIC=31577.151, Time=0.77 sec
 ARIMA(0,1,1)(0,0,1)[12] intercept   : AIC=31577.665, Time=0.77 sec
 ARIMA(0,1,0)(0,0,0)[12]             : AIC=31581.788, Time=0.03 sec
 ARIMA(2,1,2)(0,0,1)[12] intercept   : AIC=31568.878, Time=3.42 sec
 ARIMA(2,1,2)(1,0,0)[12] intercept   : AIC=31568.657, Time=3.26 sec
 ARIMA(2,1,2)(2,0,1)[12] intercept   : AIC=31565.510, Time=12.26 sec
 ARIMA(2,1,2)(2,0,0)[12] intercept   : AIC=31563.681, Time=10.37 sec
 ARIMA(1,1,2)(2,0,0)[12] intercept   : AIC=31561.131, Time=9.23 sec
 ARIMA(1,1,2)(1,0,0)[12] intercept   : AIC=31566.589, Time=2.83 sec
 ARIMA(1,1,2)(2,0,1)[12] intercept   : AIC=31562.923, Time=11.96 sec
 ARIMA(1,1,2)(1,0,1)[12] intercept   : AIC=31564.004, Time=6.42 sec
 ARIMA(0,1,2)(2,0,0)[12] intercept   : AIC=31572.453, Time=3.56 sec
 A

c:\Users\milen\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\milen\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
